<div style="font-size: 3em; font-weight: bold; text-align: center;">Transformers Anatomy</div>
<div style="text-align: right; font-size: 24px; margin-right: 10px;">Guillermo Díaz Aguado</div>

# 1 La Arquitectura de los transformadores
Los transformadores originalmente se basaban en la estructura de los *encoder-decoder*. 

**Encoder**:
* Convierte una **secuencia de tokens de entrada** en una **secuencia de vectores embebidos**, estos últimos suelen ser llamados como **hidden state** o **contexto**.

**Decoder**
* Usa el **hiden state** del **encoder** para generar iterativamente una **secuencia de tokens de salida**.

<div style="text-align:center;">
    <img src="Imagenes/Arquitectura encoder-decoder.png" width="40%">
</div>

Aunque existan cientos de arquitecturas distintas para los transformadores, muchas de ellas pertenecen a una de las siguientes clases:

*Encoder-only*:  
&nbsp;&nbsp;&nbsp;&nbsp; Estos modelos convierten una secuencia de entrada en una representación numerica. Donde más se ve es en *text classification*.

*Decoder-only*:  
&nbsp;&nbsp;&nbsp;&nbsp; Suelen ser usados para autocompletar texto mediante la predicción de la palabra más probable.

*Encoder-Decoder*:  
&nbsp;&nbsp;&nbsp;&nbsp; Encajan en tareas de traducción 



## 1. 2 El Encoder
El **encoder** consiste en muchas capas apiladas una al lado de otra. Simplemente cada capa recide una *secuencia de tokens embebidos* y se las pasa a la siguiente capa mediante las siguientes *sub-capas*:
* Una capa **multi-head self-attention**: sirve para que preste **atención** a diferentes partes de la secuencia al mismo tiempo y de formas distintas.
    * *Self attention*: implica que cada elemento se relacione a los demás de una manera.
    * *Multi-head*: genera varias relaciones en paralelo para que aprenda de forma más rica y variada. 
* Una capa **fully connected feed forward** que se aplica a cada entrada.  

<div style="text-align:center;">
    <img src="Imagenes/encoder.png" width="40%">
</div>

<div style="border: 10px solid black; padding: 10px; width:100%">
    El rol principal del Encoder es actualizar las entradas embebidas para producir representaciones con más información contextual.
</div>


### 1.2.1 Self-Attention
Es un mecanismo que permite a las redes neuronales <u> asignar diferentes pesos o "attention"</u> a cada elemento en la secuencia. 
Lo que se hace es dada una secuencia de *token embebidos* $[x_1, \ldots, x_n]$, el proceso de **self-attention** produce una nueva secuencia de *tokens embebidos* $[x'_1, \ldots, x'_n]$ donde cada $x'_i$ es una combinación lineal de todos los tokens $x_j$. Estos embeddings se llaman **contextualized embeddings**.
$$
x'_i = \sum^n_{j=1}w_{ji}x_j
$$

La idea principal de este proceso es que en vez de tener unos tokens de entrada que siempre vayan a tener los mismos valores, tengamos unos tokens que cambien su valor en función del contexto. Si hablamos de "apple" en un entorno financiero, se referirá a la compañía y no a la fruta.

Los coeficientes $w_{ji}$ son llamados **attention weights** y <u>están normalizados</u>, lo que significa que $\sum w{ji} =1 $

#### Formas de implementar Self-Attention
##### Scaled dot-product attention
<div style="text-align:center;">
    <img src="Imagenes/dot_product_attention.png" width="40%">
</div>
1. Proyectamos cada uno de los tokens embebidos en 3 dimensiones, a cada proyección la llamaremos *query*, *key* y *value*.
2. Computación de los **attention scores**. Determinamos como se relacionan los vectores *query* y *key* usando una **similarity function**. Para la **Scaled dot-product attention** usaremos como función de similaridad el *dot product* de las matrices. Aquellas *query* y *keys* que sean similiares tendrán un valor alto, y las que no sean similares tendrán un valor bajo. El valor que nos da esta función se llama **attention scores**.
3. Computación de los **pesos/weights**. La función *dot-product* puede devolver valores muy dispares y muy altos para arreglar esto se realiza:
    * *attention scores* son multiplicados por un factor de escalado para <u>normalizar la varianza</u>
    * Después se normaliza con una *softmax* para que la suma de los pesos sea 1. 
4. Actualizamos los tokens embebidos. Una vez tenemos los *pesos* y con el vector *values*, usamos estos valores para sacar los **contextualized tokens** de la siguiente forma:
$$
x'_i = \sum^n_{j=1}w_{ji}v_j
$$

Como podemos entender los vectores **query**, **key** y **value** usando la siguiente analogía:
Imagina que estas en el supermercado 
    * Tienes una lista con todos los ingredientes de tu cena -> *query*
    * Las etiquetas del supermercado -> *keys*


### 1.2.2 Multi-headed attention
Como hemso explicado antes, para el proceso de *self-attention* realizamos 3 transformaciones linealmente independientes para generar los vectores de *key*, *query* y *value*. Estas transformaciones permiten a la capa de *attention* que se enfoque en un aspecto semántico.

Pero sería beneficioso realizar estas transformaciones con otras proyecciones lineales para sacar distintos aspectos semánticos, cada una de estas proyecciones se llama **attention head**. Al realizar muchas *attention head* tendremos como resultado una **multihead attention layer**.

Así cada "cabeza" se puede enfocar en algo: tal vez una se fije en la interacción sustantivo-verbo, otra se centre en los adjetivos...

Pero, ¿Como lo podemos hacer?:
1. Inicializamos 3 capas linealmentes independientes a las que aplicamos multiplicación matricial a los *vectores embebidos*, donde conseguiremos tensores con la forma *[batch_size, seq_len, head_dim]*
2. **head_dim** es el número de dimensiones sobre la que estamos proyectando.   
   **embed_dim** es el número de dimensiones del *vector embebido*.
3. Realmente podemos hacer que **head_dim**=**embed_dim** pero en la práctica lo mejor es que **head_dim** sea un multiplo de **embed_dim**, para facilitar cálculos. 
4. Ahora que tenemos todas las **single attention head**, tan solo tenemos que concatenarlas para implementar la **multihead attention layer**. En este caso conseguiremos unos tensores con la forma *[batch_size, seq_len, hidden_dim]*


<div style="text-align:center;">
    <img src="Imagenes/multihead_attention.png" width="40%">
</div>

### 1.2.3 The Feed Forward Layer
La **Feed Forward Layer** en la estructura del *encoder-decoder* es simplemente **dos capas de redes neuronales totalmente conectadas** (two-layer fully connected neural network), pero con un giro: <u> en vez de procesar toda la secuencia de *embeddings* como un solo vector, procesa cada *embedding* de forma **independiente** </u>. Por esta razón se refiere a este capa como **position-wise feed-forward layer**. 

Según el libro *Natural LAnguage Processing with Transformers* deberiamos usar la siguiente configuración para estas capas:
* El tamaño de la capa oculta de la primera capa debe ser x4 el tamaño de los *embeddings*.
* Se usarán como funciones de activación la **GELU**. 

### 1.2.4 Normalization layer and Skip Connections
Ahora que sabemos todas las capas que calculan los datos en un **Transformador**, debemos saber donde colocar las **Capas de Normalización** y las **Skip connections**. 

**Capas de Normalización**  
Son capas cuya función es que cada entrada en el batch tenga:
* media = 0
* varianza = 1

**Skip connections**--> **residual otuputs**
Es cuando tomamos la entrada original de un bloque (`x`) y lo sumamos con la salida del bloque transformado (`f(x)`), así evitamos que las redes se **desentrenen**. 


A la hora de colocarlos hay dos formas predominantes:
* *Post layer normalization*  
    Esta es la distribución usada en el *Transformer paper*. Coloca las capas de normalización después de sumar la entrada original con el bloque transformado.  
    $$output = LayerNorm(f(x) + x)$$   
    De esta forma es engañoso/dificil (tricky) entrenar el modelo, ya que el gradiente suele diverger, por eso se suele usar un concepto llamado *learning rate warm-up* que consiste en incrementar gradualmente la tasa de aprendizaje desde un valor pequeño a un máximo. 

* *Pre layer normalization*  
    Esta la distribución más común. Realiza la normalización antes de procesar el bloque transformador.  
    $$ output = x + f(LayerNorm(x))$$

Aquí muestro una imagen, pero creo que es más entendible con las formulas
<div style="text-align:center;">
    <img src="Imagenes/layer_normalization.png" width="40%">
</div>
